In [1]:
from pathlib import Path
from polyquant.data.schema import load_schema
from polyquant.data.normalize import load_feature_scaler
from polyquant.data.datasets.tabular import TabularParquetIterable
import torch

ROOT = Path(r"C:\Users\nimro\PolyQuant")
DATASET_ROOT = ROOT / "data" / "features_dataset"
SCALER_PATH = DATASET_ROOT / "train_scaler.json"

schema = load_schema(DATASET_ROOT)
scaler = load_feature_scaler(SCALER_PATH, schema.feature_cols, schema.no_scale_cols)
ds = TabularParquetIterable(
    split_dir=DATASET_ROOT / "train",
    feature_cols=schema.feature_cols,
    scaler=scaler,
    batch_size=8192,
    shuffle_files=True,
    shuffle_rowgroup=True,
)

total = 0
pos = 0
for i, batch in enumerate(ds):
    y = batch["y"]
    total += y.numel()
    pos += int(y.sum())
    if total >= 500_000:  # enough
        break

print("positive rate ~", pos / total)


positive rate ~ 0.5141889080191144


In [2]:
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from polyquant.data.schema import load_schema
from polyquant.data.normalize import load_feature_scaler
from polyquant.data.datasets.tabular import TabularParquetIterable
from polyquant.models.mlp import MLP

ROOT = Path(r"C:\Users\nimro\PolyQuant")
DATASET_ROOT = ROOT / "data" / "features_dataset"
SCALER_PATH = DATASET_ROOT / "train_scaler.json"

schema = load_schema(DATASET_ROOT)
scaler = load_feature_scaler(SCALER_PATH, schema.feature_cols, schema.no_scale_cols)

ds = TabularParquetIterable(
    split_dir=DATASET_ROOT / "train",
    feature_cols=schema.feature_cols,
    scaler=scaler,
    batch_size=8192,
    shuffle_files=True,
    shuffle_rowgroup=True,
    seed=123,
)
loader = DataLoader(ds, batch_size=None, num_workers=0)

device = torch.device("cuda")
model = MLP(in_dim=len(schema.feature_cols)).to(device)
loss_fn = nn.BCEWithLogitsLoss()

batch = next(iter(loader))
x = batch["x"].to(device)
y = batch["y"].to(device)

with torch.no_grad():
    logits = model(x)
    loss = loss_fn(logits, y)

print("initial BCE (no training):", float(loss))


initial BCE (no training): 0.7049640417098999
